In [1]:
import numpy as np
from numpy import linalg
import os
import json

In [2]:
def gaussian_entropy(cov_matrix):
    cov_matrix = np.atleast_2d(cov_matrix)
    n = len(cov_matrix)
    det_cov = linalg.det(cov_matrix)
    return 1/2 * np.log2((2*np.pi*np.exp(1))**n * det_cov)

def teoric_TC(cov_matrix):
    n = len(cov_matrix)
    # H(X1,...,Xn)
    H_x = gaussian_entropy(cov_matrix)
    # H(Xi)
    H_xi = [gaussian_entropy(cov_matrix[i, i]) for i in range(n)]
    # sum(H(Xi))
    sum_H_xi = np.sum(H_xi)
    # TC
    TC = sum_H_xi - H_x
    return TC

def teoric_DTC(cov_matrix):
    n = len(cov_matrix)
    # H(X1,...,Xi-1,Xi+1,...,Xn)
    H_xis_ = [gaussian_entropy(np.delete(np.delete(cov_matrix, j, axis=1), j, axis=0)) for j in range(n)] 
    # sum(H(X1,...,Xi-1,Xi+1,...,Xn))
    sum_H_xi_ = np.sum(H_xis_) 
    # H(X1,...,Xn)
    H_x = gaussian_entropy(cov_matrix)
    # (1-n) * H(X1,...,Xn)
    prod_H_x = (1-n) * H_x
    # DTC
    DTC = prod_H_x + sum_H_xi_
    return DTC

def teoric_oinfo(cov_matrix):
    oinfo = teoric_TC(cov_matrix) - teoric_DTC(cov_matrix)
    return oinfo

In [3]:
def makeSamples(parametersList, repetition_number, save_npy=False):    
    i=1
    for parameters in parametersList:
        os.mkdir('./muestras/gaussian_'+str(i))
        cov = parameters['cov']
        dim = len(cov)
        TC_teoric = teoric_TC(cov)
        DTC_teoric = teoric_DTC(cov)
        O_teoric = TC_teoric-DTC_teoric
        header = f" {parameters['n_samples']} samples \n Cov = \n{cov}\n TC_teoric = {TC_teoric}\n DTC_teoric = {DTC_teoric}\n O_teoric = {O_teoric}"
        for j in range(1,repetition_number+1):
            np.random.seed(j)
            normal = np.random.multivariate_normal(np.zeros(dim), parameters['cov'], parameters['n_samples'])
            if (save_npy):
                np.save(f'./muestras/gaussian_{i}/samples{j}.npy',normal)
            else:
                np.savetxt(f'./muestras/gaussian_{i}/samples{j}.txt',normal, header=header, comments='%')
        i+=1

In [4]:
def makeJson(parametersList):
    gaussians={}
    for i, parameters in enumerate(parametersList):
        cov = parameters['cov']
        n_samples = parameters['n_samples']
        TC_teoric_value = teoric_TC(cov)
        DTC_teoric_value = teoric_DTC(cov)
        O_teoric_value = TC_teoric_value-DTC_teoric_value
        gaussians[f"gaussian_{i+1}"] = {
            "cov": cov.tolist(),
            "TC": TC_teoric_value,
            "DTC": DTC_teoric_value,
            "O": O_teoric_value
        }
    with open("./muestras/index.json", "w") as f:
        json.dump(gaussians, f)

In [5]:
def crearMatriz(N, seed = 1):
    np.random.seed(seed)
    covmat = np.random.rand(N,N)
    covmat = covmat @ covmat.transpose()
    return covmat

In [6]:
"""for i in range(1000):
    cov=crearMatriz(10,seed=i)
    o = teoric_oinfo(cov)
    if abs(o)<0.01:
        print(o,i)"""

'for i in range(1000):\n    cov=crearMatriz(10,seed=i)\n    o = teoric_oinfo(cov)\n    if abs(o)<0.01:\n        print(o,i)'

In [7]:
n_samples = 100000
parametersList = [
{
    "n_samples": n_samples,
    "cov": crearMatriz(3, seed = 98)#Negativo
},
{
    "n_samples": n_samples,
    "cov": crearMatriz(3, seed = 746)#Positivo
},
{
    "n_samples": n_samples,
    "cov": crearMatriz(3, seed = 331)#Cercano a cero
},
{
    "n_samples": n_samples,
    "cov": crearMatriz(4, seed = 222)#Negativo
},
{
    "n_samples": n_samples,
    "cov": crearMatriz(4, seed = 569)#Positivo
},
{
    "n_samples": n_samples,
    "cov": crearMatriz(4, seed = 359)#Cercano a cero
},
{
    "n_samples": n_samples,
    "cov": crearMatriz(10, seed = 33)#Negativo
},
{
    "n_samples": n_samples,
    "cov": crearMatriz(10, seed = 24)#Positivo
},
{
    "n_samples": n_samples,
    "cov": crearMatriz(10, seed = 775)#Cercano a cero
}
]

In [8]:
os.mkdir('./muestras')
makeJson(parametersList)
makeSamples(parametersList,100,save_npy=False)
#makeSamples(parametersList,100,save_npy=True)